In [274]:
from pyOrbfit.Orbit import Orbit
import pandas as pd
import ephem
import os
import numpy as np
import easyaccess as ea

In [288]:
def giveTable(sid, canid): #does this include only Y3 stuff?
    db=ea.connect(section='desoper')
    query="select ra, dec, expnum, a from scheibne.searchexps where canid = '"+canid+"' and searchId =" +str(sid)
    result=db.query_to_pandas(query)
    ra=np.degrees(ephem.hours(result['RA'][0]))
    dec=np.degrees(ephem.degrees(result['DEC'][0]))
    expnum=result['EXPNUM'][0]
    width=.005#3*result['A'][0]/3600 #this could be adjusted
    print "search RA: "+str(ra), 'search DEC'+str(dec), "search width"+str(width)
    SEquerry="select distinct o.EXPNUM,c.CCDNUM,e.MJD_OBS,o.FILENAME,o.RA,o.DEC,o.FLUX_AUTO,o.FLUXERR_AUTO,o.FLAGS, \
    o.IMAFLAGS_ISO,o.BAND,z.fgcm_zpt, z.fgcm_zpterr, z.flag, z.calnac, z.blacklisted from prod.proctag t,prod.catalog \
    c,prod.se_object o,prod.exposure e,prod.finalcut_eval f,ERYKOFF.Y3A1_FGCM_Y1Y2Y3_V1_0@DESSCI z where tag= \
    'Y3A1_FINALCUT' and t.pfw_attempt_id=c.pfw_attempt_id and c.filename=o.filename and o.expnum=e.expnum and \
    z.pfw_attempt_id=t.pfw_attempt_id and z.expnum=e.expnum and z.ccd=c.ccdnum and f.expnum=e.expnum and \
    f.accepted='True' and o.BAND!='Y' and o.ra between "+str(ra-width)+" and "+str(ra+width)+" and o.dec \
    between "+str(dec-width)+" and "+str(dec+width)+" and e.exptime>80.0 order by o.EXPNUM,c.CCDNUM"
    return db.query_to_pandas(SEquerry), expnum

In [94]:
good19Table,exp=giveTable(4,'S10Y2NF_good_19')

Connecting to DB ** desoper ** ...
22.5272916667 -3.82858333333 0.001075
select distinct o.EXPNUM,c.CCDNUM,e.MJD_OBS,o.FILENAME,o.RA,o.DEC,o.FLUX_AUTO,o.FLUXERR_AUTO,o.FLAGS,     o.IMAFLAGS_ISO,o.BAND,z.fgcm_zpt, z.fgcm_zpterr, z.flag, z.calnac, z.blacklisted from prod.proctag t,prod.catalog     c,prod.se_object o,prod.exposure e,prod.finalcut_eval f,ERYKOFF.Y3A1_FGCM_Y1Y2Y3_V1_0@DESSCI z where tag=     'Y3A1_FINALCUT' and t.pfw_attempt_id=c.pfw_attempt_id and c.filename=o.filename and o.expnum=e.expnum and     z.pfw_attempt_id=t.pfw_attempt_id and z.expnum=e.expnum and z.ccd=c.ccdnum and f.expnum=e.expnum and     f.accepted='True' and o.BAND!='Y' and o.ra between 22.5262166667 and 22.5283666667 and o.dec     between -3.82965833333 and -3.82750833333 and e.exptime>80.0 order by o.EXPNUM,c.CCDNUM


In [273]:
def findTrans(frame,expnum, threshold=.000010):
    ourDetections=frame[frame['EXPNUM']==expnum]
    otherDetects=frame[frame['EXPNUM']!=expnum]
    transients=pd.DataFrame()
    for d in ourDetections.index:
        ra=ourDetections['RA'][d]
        dec=ourDetections['DEC'][d]
        mjd=ourDetections['MJD_OBS'][d]
        stationary=False
        for i in otherDetects.index:
            ora=otherDetects['RA'][i]
            odec=otherDetects['DEC'][i]
            omjd=otherDetects['MJD_OBS'][i]
            if  abs(ra-ora)<threshold and abs(dec-odec)<threshold and abs(mjd-omjd)>1: #last condition is a time displacement
                stationary=True
                otherDetects=otherDetects.drop(i)
        if stationary==False:
            transients=transients.append(ourDetections.ix[d:d])
    return transients

In [262]:
myTransients=findTrans(good19Table,exp)

In [258]:
good19Table

,EXPNUM,CCDNUM,MJD_OBS,FILENAME,RA,DEC,FLUX_AUTO,FLUXERR_AUTO,FLAGS,IMAFLAGS_ISO,BAND,FGCM_ZPT,FGCM_ZPTERR,FLAG,CALNAC,BLACKLISTED
0,372594,30,56959.257835,D00372594_i_c30_r2182p01_red-fullcat.fits,22.527079,-3.828513,9406.830078,566.562012,0,2048,i,31.600398,0.005641,0,0,0
1,372595,9,56959.259216,D00372595_r_c09_r2182p02_red-fullcat.fits,22.527090,-3.828513,9055.889648,472.345001,0,0,r,31.658712,0.006932,0,0,0


In [ ]:
def fit_orbit(df_obs):
    df_obs = df_obs.ix[['#' not in row['DATE_OBS'] for ind, row in df_obs.iterrows()]]   # filter comment lines
    nobs = len(df_obs)
    ralist = [ephem.hours(r) for r in df_obs['RA'].values]
    declist = [ephem.degrees(r) for r in df_obs['DEC'].values]
    datelist = [ephem.date(d) for d in df_obs['DATE_OBS'].values]
    obscode = np.ones(nobs, dtype=int)*807
    orbit = Orbit(dates=datelist, ra=ralist, dec=declist, obscode=obscode, err=0.15)
    return orbit

In [270]:
def evalChi(transFrame,canid):
    db=ea.connect(section='desoper')
    query="select distinct ra, dec, date_obs from scheibne.OBJECTS inner join scheibne.linker on \
objects.objid=linker.objid where linker.canid='"+canid+"'"
    oldList=db.query_to_pandas(query)
    nobs = len(oldList)
    oldra = [ephem.hours(r) for r in oldList['RA'].values]
    olddec =[ephem.degrees(r) for r in oldList['DEC'].values]
    olddate = [ephem.date(d) for d in oldList['DATE_OBS'].values]
    obscode = np.ones(nobs, dtype=int)*807
    oldFit=Orbit(dates=olddate, ra=oldra, dec=olddec, obscode=obscode, err=0.15)
    oldChi=oldFit.chisq
    oldNdof=oldFit.ndof
    oldMetric=oldChi/oldNdof
    transFrame["ChisqPerNdof"]=0*np.ones(len(transFrame))
    transFrame["OldChiPerN"]=oldMetric
    for i in transFrame.index:
        newra=oldra+[ephem.hours(np.radians(transFrame['RA'][i]))]
        newdec=olddec+[ephem.degrees(np.radians(transFrame['DEC'][i]))]
        newdate=olddate+[ephem.date(transFrame['MJD_OBS'][i]-15019.5)]
        newobscode=np.ones(nobs+1,dtype=int)*807
        newFit=Orbit(dates=newdate, ra=newra, dec=newdec, obscode=newobscode, err=0.15)
        newChi=newFit.chisq
        newNdof=newFit.ndof
        newMetric=newChi/newNdof
        transFrame["ChisqPerNdof"][i]=newMetric
    return transFrame

In [265]:
WChiTrans=evalChi(myTransients,'S10Y2NF_good_19')

Connecting to DB ** desoper ** ...
0.343869848831


In [266]:
WChiTrans

,EXPNUM,CCDNUM,MJD_OBS,FILENAME,RA,DEC,FLUX_AUTO,FLUXERR_AUTO,FLAGS,IMAFLAGS_ISO,BAND,FGCM_ZPT,FGCM_ZPTERR,FLAG,CALNAC,BLACKLISTED,ChisqPerNdof,OldChiPerN
0,372594,30,56959.257835,D00372594_i_c30_r2182p01_red-fullcat.fits,22.527079,-3.828513,9406.830078,566.562012,0,2048,i,31.600398,0.005641,0,0,0,0.34387,0.557831


In [263]:
myTransients

,EXPNUM,CCDNUM,MJD_OBS,FILENAME,RA,DEC,FLUX_AUTO,FLUXERR_AUTO,FLAGS,IMAFLAGS_ISO,BAND,FGCM_ZPT,FGCM_ZPTERR,FLAG,CALNAC,BLACKLISTED
0,372594,30,56959.257835,D00372594_i_c30_r2182p01_red-fullcat.fits,22.527079,-3.828513,9406.830078,566.562012,0,2048,i,31.600398,0.005641,0,0,0


In [267]:
def CalcMag(transFrame):
    transFrame["MAG"]=0*np.ones(len(transFrame))
    for i in transFrame.index:
        fluxAuto=transFrame["FLUX_AUTO"][i]
        zp=transFrame["FGCM_ZPT"][i]
        transFrame['MAG'][i]=zp-2.5*np.log10(fluxAuto)
    return transFrame

In [268]:
CalcMag(WChiTrans)

,EXPNUM,CCDNUM,MJD_OBS,FILENAME,RA,DEC,FLUX_AUTO,FLUXERR_AUTO,FLAGS,IMAFLAGS_ISO,BAND,FGCM_ZPT,FGCM_ZPTERR,FLAG,CALNAC,BLACKLISTED,ChisqPerNdof,OldChiPerN,MAG
0,372594,30,56959.257835,D00372594_i_c30_r2182p01_red-fullcat.fits,22.527079,-3.828513,9406.830078,566.562012,0,2048,i,31.600398,0.005641,0,0,0,0.34387,0.557831,21.66679


In [284]:
def pointInspector(sid,canid):
    allPoints,exp=giveTable(sid,canid)
    trans=findTrans(allPoints,exp,.000001)
    WChi=evalChi(trans,canid)
    WMag=CalcMag(WChi)
    for i in WMag.index:
        mystring= "RA: "+str(ephem.hours(np.radians(WMag['RA'][i])))+" DEC: "+str(ephem.degrees(np.radians(WMag['DEC'][i])))+" OldChiPerN: "+str(WMag['OldChiPerN'][i])+" NewChiPerN: "+str(WMag['ChisqPerNdof'][i])+" MAG: "+str(WMag['MAG'][i])
        print mystring

In [278]:
pointInspector(4,'S10Y2NF_good_19') #not sure why mag is an integer, not sure why nic chi is 0

Connecting to DB ** desoper ** ...
search RA: 22.5272916667 search DEC-3.82858333333 search width0.001075
Connecting to DB ** desoper ** ...
RA: 1:30:06.50 DEC: -3:49:42.6 OldChiPerN: 0.557831459569 NewChiPerN: 0.343869848831 MAG: 21.6667895558


In [289]:
pointInspector(1,'S10Y2NF_good_2')

Connecting to DB ** desoper ** ...
search RA: 13.0109166667 search DEC4.55236111111 search width0.005
Connecting to DB ** desoper ** ...
RA: 0:52:01.52 DEC: 4:33:21.2 OldChiPerN: 1.35186320319 NewChiPerN: 7303.07705858 MAG: 23.2515301591
RA: 0:52:02.58 DEC: 4:33:08.2 OldChiPerN: 1.35186320319 NewChiPerN: 3.38958459665 MAG: 23.0570554971


In [290]:
pointInspector(2,'S10Y2NF_good_2')

Connecting to DB ** desoper ** ...
search RA: 13.0108333333 search DEC4.55233333333 search width0.005
Connecting to DB ** desoper ** ...
RA: 0:52:02.43 DEC: 4:33:07.5 OldChiPerN: 1.35186320319 NewChiPerN: 2.69225766265 MAG: 23.0653909649


In [291]:
pointInspector(6,'S10Y2NF_good_2')

Connecting to DB ** desoper ** ...
search RA: 12.4766666667 search DEC4.2445 search width0.005
Connecting to DB ** desoper ** ...
RA: 0:49:54.48 DEC: 4:14:38.9 OldChiPerN: 1.35186320319 NewChiPerN: 23.0904159781 MAG: 22.5535531358
RA: 0:49:54.51 DEC: 4:14:33.4 OldChiPerN: 1.35186320319 NewChiPerN: 465.154250322 MAG: 22.6282561471


In [292]:
pointInspector(1,'S13Y2NF_good_2')

Connecting to DB ** desoper ** ...
search RA: 42.2136666667 search DEC-9.39791666667 search width0.005
Connecting to DB ** desoper ** ...
RA: 2:48:50.64 DEC: -9:24:05.6 OldChiPerN: 0.785780052032 NewChiPerN: 3.9156589691 MAG: 21.0369169753
RA: 2:48:51.46 DEC: -9:23:49.9 OldChiPerN: 0.785780052032 NewChiPerN: 0.78627080783 MAG: 21.2154934568


In [293]:
pointInspector(2,'S13Y2NF_good_2')

Connecting to DB ** desoper ** ...
search RA: 42.2136666667 search DEC-9.39797222222 search width0.005
Connecting to DB ** desoper ** ...
RA: 2:48:50.64 DEC: -9:24:05.6 OldChiPerN: 0.785780052032 NewChiPerN: 3.85690213819 MAG: 20.5961645195
RA: 2:48:51.45 DEC: -9:23:50.1 OldChiPerN: 0.785780052032 NewChiPerN: 0.77776911913 MAG: 20.3913061514


In [294]:
pointInspector(3,'S13Y2NF_good_2')

Connecting to DB ** desoper ** ...
search RA: 41.4817916667 search DEC-9.93294444444 search width0.005
Connecting to DB ** desoper ** ...
RA: 2:45:54.66 DEC: -9:56:09.8 OldChiPerN: 0.785780052032 NewChiPerN: 1090.1215733 MAG: 22.1218196454
RA: 2:45:55.12 DEC: -9:55:44.4 OldChiPerN: 0.785780052032 NewChiPerN: 860.595640476 MAG: 23.2859624613
RA: 2:45:55.44 DEC: -9:56:09.7 OldChiPerN: 0.785780052032 NewChiPerN: 413.06557204 MAG: 22.3339336571
RA: 2:45:55.62 DEC: -9:55:58.5 OldChiPerN: 0.785780052032 NewChiPerN: 0.676433740046 MAG: 20.0926702602
RA: 2:45:55.77 DEC: -9:55:56.0 OldChiPerN: 0.785780052032 NewChiPerN: 38.2572788425 MAG: 23.3961635862
RA: 2:45:55.81 DEC: -9:56:13.1 OldChiPerN: 0.785780052032 NewChiPerN: 669.998345582 MAG: 23.1369669595
RA: 2:45:55.97 DEC: -9:56:15.3 OldChiPerN: 0.785780052032 NewChiPerN: 933.673704754 MAG: 23.9131330781
RA: 2:45:56.65 DEC: -9:56:06.5 OldChiPerN: 0.785780052032 NewChiPerN: 903.58462306 MAG: 21.2149268658


In [295]:
pointInspector(4,'S13Y2NF_good_2')

Connecting to DB ** desoper ** ...
search RA: 41.48175 search DEC-9.93294444444 search width0.005
Connecting to DB ** desoper ** ...
RA: 2:45:55.61 DEC: -9:55:58.5 OldChiPerN: 0.785780052032 NewChiPerN: 0.744230470735 MAG: 20.9902189858
RA: 2:45:55.98 DEC: -9:56:15.1 OldChiPerN: 0.785780052032 NewChiPerN: 916.362798263 MAG: 23.655247215
RA: 2:45:56.66 DEC: -9:56:06.4 OldChiPerN: 0.785780052032 NewChiPerN: 921.602184066 MAG: 22.1610246091


In [296]:
pointInspector(2,'S118Y3NF_good_0')

Connecting to DB ** desoper ** ...
search RA: 32.4138333333 search DEC-27.3981944444 search width0.005
Connecting to DB ** desoper ** ...
RA: 2:09:38.70 DEC: -27:24:06.4 OldChiPerN: 0.840616854523 NewChiPerN: 654.303530286 MAG: 21.9672803629
RA: 2:09:39.75 DEC: -27:24:08.8 OldChiPerN: 0.840616854523 NewChiPerN: 740.437610199 MAG: 21.7652566561
RA: 2:09:40.11 DEC: -27:24:08.4 OldChiPerN: 0.840616854523 NewChiPerN: 926.459704176 MAG: 20.8072505967
RA: 2:09:40.41 DEC: -27:23:38.5 OldChiPerN: 0.840616854523 NewChiPerN: 1217.64531873 MAG: 20.400719809


In [212]:
def findPoints(ra, dec):
    RA = np.degrees(ephem.hours(ra))
    DEC= np.degrees(ephem.degrees(dec))
    print RA
    print DEC
    db=ea.connect(section='desoper')
    query="select distinct o.EXPNUM,c.CCDNUM,e.MJD_OBS,e.date_obs,o.FILENAME,o.RA,o.DEC,o.FLUX_AUTO,o.FLUXERR_AUTO, \
o.FLAGS,o.IMAFLAGS_ISO,o.BAND from prod.proctag t,prod.catalog c,prod.se_object o,prod.exposure e,prod.finalcut_eval f \
where tag= 'Y3A1_FINALCUT' and t.pfw_attempt_id=c.pfw_attempt_id and c.filename=o.filename and o.expnum=e.expnum and \
f.expnum=e.expnum and f.accepted='True' and o.BAND!='Y' and o.ra between "+str(RA)+"-0.008 and "+str(RA)+"+0.008 and \
o.dec between "+str(DEC)+"-0.008 and "+str(DEC)+"+0.008 and e.exptime>80.0 order by o.EXPNUM,c.CCDNUM"
    results=db.query_to_pandas(query)
    return results

In [231]:
incTable=findPoints('3:40:01.59','-30:34:51.8') #403066 

55.006625
-30.5810555556
Connecting to DB ** desoper ** ...


In [232]:
incTable.sort_values(by=['RA'])

,EXPNUM,CCDNUM,MJD_OBS,DATE_OBS,FILENAME,RA,DEC,FLUX_AUTO,FLUXERR_AUTO,FLAGS,IMAFLAGS_ISO,BAND
78,403064,60,57048.142514,2015-01-26T03:25:13.209335,D00403064_i_c60_r2284p01_red-fullcat.fits,55.001393,-30.577361,7410.729980,780.109009,0,0,i
60,401211,60,57043.157016,2015-01-21T03:46:06.224029,D00401211_r_c60_r2284p01_red-fullcat.fits,55.001394,-30.577376,4554.810059,565.356018,0,0,r
0,276397,59,56678.111758,2014-01-21T02:40:55.914112,D00276397_z_c59_r2299p01_red-fullcat.fits,55.001413,-30.577303,6301.470215,868.343018,0,0,z
50,401210,40,57043.155657,2015-01-21T03:44:08.741861,D00401210_i_c40_r2284p01_red-fullcat.fits,55.001422,-30.577336,7235.060059,714.668030,0,0,i
9,276736,59,56679.135998,2014-01-22T03:15:50.207771,D00276736_i_c59_r2299p01_red-fullcat.fits,55.001424,-30.577289,7122.009766,786.171997,0,0,i
96,403066,40,57048.145225,2015-01-26T03:29:07.423310,D00403066_r_c40_r2284p01_red-fullcat.fits,55.001431,-30.577314,3657.330078,478.216003,0,0,r
41,395553,40,57030.152706,2015-01-08T03:39:53.797618,D00395553_z_c40_r2283p01_red-fullcat.fits,55.001446,-30.577293,7903.330078,997.911987,0,0,z
18,278487,59,56685.087311,2014-01-28T02:05:43.712472,D00278487_r_c59_r2300p01_red-fullcat.fits,55.001461,-30.577352,3024.639893,439.524994,0,0,r
61,401211,60,57043.157016,2015-01-21T03:46:06.224029,D00401211_r_c60_r2284p01_red-fullcat.fits,55.001841,-30.588221,1986.579956,378.626007,0,0,r
19,278487,59,56685.087311,2014-01-28T02:05:43.712472,D00278487_r_c59_r2300p01_red-fullcat.fits,55.001846,-30.588164,1767.569946,343.803986,0,0,r


In [251]:
transFound=findTrans(incTable,403066,.000100)

TypeError: evalChi() takes exactly 2 arguments (1 given)

In [236]:
oldFrame=pd.read_csv('good_2.csv')

In [243]:
def fit_orbit(df_obs):
    df_obs = df_obs.ix[['#' not in row['date'] for ind, row in df_obs.iterrows()]]   # filter comment lines
    nobs = len(df_obs)
    ralist = [ephem.hours(r) for r in df_obs['ra'].values]
    declist = [ephem.degrees(r) for r in df_obs['dec'].values]
    datelist = [ephem.date(d) for d in df_obs['date'].values]
    obscode = np.ones(nobs, dtype=int)*807
    orbit = Orbit(dates=datelist, ra=ralist, dec=declist, obscode=obscode, err=0.15)
    return orbit

In [244]:
def addToOrbfit(oldList,mjd,ra,dec,expnum):
    newlist=oldList.append({'expnum':expnum, 'date':str(ephem.date(mjd-15019.5)),'ra': str(ephem.hours(np.radians(ra))) , 'dec':str(ephem.degrees(np.radians(dec)))},ignore_index=True)
    newOrb=fit_orbit(newlist)
    print newOrb.chisq/newOrb.ndof
    return newOrb

In [246]:
newFit=addToOrbfit(oldFrame, 57048.143874, 55.006631, -30.581068,403064)

0.557633684918


In [248]:
oldFit=fit_orbit(oldFrame)

In [249]:
oldFit.chisq/oldFit.ndof

0.6018014276342941

In [67]:
myPandas=pd.read_csv('good_2.csv')

In [23]:
myPandas.drop(myPandas.index[[3]])

,fate,ra,dec,expnum,exptime,band,ccd,mag,ml_score,objid,fakeid
0,12/24/14 4:04,35:16.8,-14:18:09.9,390941,90,r,22,23.87,0.780,25189745,0
1,12/16/14 4:41,35:43.6,-14:21:48.9,388201,90,i,26,23.28,0.858,25564805,0
2,12/12/14 4:48,35:58.9,-14:23:17.2,386740,90,i,23,23.42,0.742,24509371,0


In [26]:
myPandas.index[0]

0

In [55]:
myPandas.ix[2:2]

,fate,ra,dec,expnum,exptime,band,ccd,mag,ml_score,objid,fakeid
2,12/12/14 4:48,35:58.9,-14:23:17.2,386740,90,i,23,23.42,0.742,24509371,0


In [69]:
for j in myPandas.index:
    print myPandas['expnum'][j]
    if myPandas['ccd'][j]==23:
        print myPandas['ccd'][j]
        print j
        myPandas=myPandas.drop(j)
    

390941
388201
386740
23
2
386739
23
3


In [70]:
myPandas

,fate,ra,dec,expnum,exptime,band,ccd,mag,ml_score,objid,fakeid
0,12/24/14 4:04,35:16.8,-14:18:09.9,390941,90,r,22,23.87,0.780,25189745,0
1,12/16/14 4:41,35:43.6,-14:21:48.9,388201,90,i,26,23.28,0.858,25564805,0


In [71]:
myPandas.index[1]

1

In [281]:
print ephem.date(56537.257498-15019.5)

2013/9/2 06:10:48


In [287]:
5.0/3600

0.001388888888888889